<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-data-and-Prep-work" data-toc-modified-id="Read-in-data-and-Prep-work-1">Read in data and Prep work</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1.1">Data</a></span></li><li><span><a href="#Prep-the-tokenizer-class" data-toc-modified-id="Prep-the-tokenizer-class-1.2">Prep the tokenizer class</a></span></li><li><span><a href="#Prep-the-&quot;Stance&quot;-label" data-toc-modified-id="Prep-the-&quot;Stance&quot;-label-1.3">Prep the "Stance" label</a></span></li><li><span><a href="#Prep-premises,-conslusions,-and-stance-labels" data-toc-modified-id="Prep-premises,-conslusions,-and-stance-labels-1.4">Prep premises, conslusions, and stance labels</a></span></li><li><span><a href="#One-hot-encode-the-stance-label-for-Neural-net-architecture" data-toc-modified-id="One-hot-encode-the-stance-label-for-Neural-net-architecture-1.5">One hot encode the stance label for Neural net architecture</a></span></li><li><span><a href="#Batch-the-Train,-Validation,-and-Test-sets" data-toc-modified-id="Batch-the-Train,-Validation,-and-Test-sets-1.6">Batch the Train, Validation, and Test sets</a></span><ul class="toc-item"><li><span><a href="#Selecting-which-labels-to-predict" data-toc-modified-id="Selecting-which-labels-to-predict-1.6.1">Selecting which labels to predict</a></span></li></ul></li></ul></li><li><span><a href="#Naive-Model-1" data-toc-modified-id="Naive-Model-1-2">Naive Model 1</a></span><ul class="toc-item"><li><span><a href="#Using-3-labels" data-toc-modified-id="Using-3-labels-2.1">Using 3 labels</a></span><ul class="toc-item"><li><span><a href="#Final-F1-score" data-toc-modified-id="Final-F1-score-2.1.1">Final F1 score</a></span></li><li><span><a href="#Predicting-on-Test-Set" data-toc-modified-id="Predicting-on-Test-Set-2.1.2">Predicting on Test Set</a></span></li></ul></li><li><span><a href="#Using-full-labels" data-toc-modified-id="Using-full-labels-2.2">Using full labels</a></span><ul class="toc-item"><li><span><a href="#Adjust-the-activation-layer" data-toc-modified-id="Adjust-the-activation-layer-2.2.1">Adjust the activation layer</a></span></li></ul></li></ul></li><li><span><a href="#Dual-Model-Approach" data-toc-modified-id="Dual-Model-Approach-3">Dual Model Approach</a></span><ul class="toc-item"><li><span><a href="#Rough-sketch-of-how-the-ideal-system-would-work" data-toc-modified-id="Rough-sketch-of-how-the-ideal-system-would-work-3.1">Rough sketch of how the ideal system would work</a></span></li><li><span><a href="#Test-set-on-models" data-toc-modified-id="Test-set-on-models-3.2">Test set on models</a></span></li></ul></li><li><span><a href="#Smart-Model-Approach" data-toc-modified-id="Smart-Model-Approach-4">Smart Model Approach</a></span></li></ul></div>

In [256]:
import pandas as pd
import numpy as np
import torch
from typing import List, Dict
from transformers import BertModel
from transformers import AutoTokenizer
from transformers import BertForPreTraining
from transformers import BertForSequenceClassification
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.metrics import f1_score

# Read in data and Prep work

## Data

In [49]:
# Read in the original data
training_arguments = pd.read_csv('./data/arguments-training.tsv',delimiter='\t')
validation_arguments = pd.read_csv('./data/arguments-validation.tsv',delimiter='\t')
test_arguments = pd.read_csv('./data/arguments-test.tsv',delimiter='\t')

# Americanize the UK "favour" (18 'in favour of' vs 2880 'in favor of')
print(training_arguments['Stance'].value_counts())
training_arguments.loc[training_arguments['Stance'] == "in favour of", 'Stance'] = "in favor of"
validation_arguments.loc[validation_arguments['Stance'] == "in favour of", 'Stance'] = "in favor of"
test_arguments.loc[test_arguments['Stance'] == "in favour of", 'Stance'] = "in favor of"

training_semlabels = pd.read_csv('./data/labels-training.tsv',delimiter='\t')
validation_semlabels = pd.read_csv('./data/labels-validation.tsv',delimiter='\t')

in favor of     2880
against         2495
in favour of      18
Name: Stance, dtype: int64


In [50]:
training_semlabels

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,A01006,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A01007,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,A01008,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5388,E08016,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5389,E08017,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,1
5390,E08018,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5391,E08019,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,1


## Prep the tokenizer class

In [51]:
class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
    
    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token
    
    def __call__(self, prem_batch: List[str], hyp_batch: List[str]) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # The two sentences deimited by the [SEP] token.
        enc = self.hf_tokenizer(
            prem_batch,
            hyp_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc

In [52]:
test_arguments.loc[:,["Conclusion", "Premise"]].values

array([['We should end affirmative action',
        'affirmative action helps with employment equity.'],
       ['We should end affirmative action',
        'affirmative action can be considered discriminatory against poor whites'],
       ['We should ban naturopathy',
        'naturopathy is very dangerous for the most vulnerable people, like children and cancer patients. people use ineffective treatments and forgo proven cures, such as antibiotics or chemo, often resulting in death.'],
       ...,
       ['We should consider Russian interests in the EU policy.',
        'It is neither in the interests of the EU nor Russia to split a long common history. The dialogue must never stop,  With respect and trade, interpersonal exchange, and further expansion of what almost ended in 2014. Here I am thinking of space exploration, and joint development of vehicles, aircraft, etc. Our time horizon must be long-term - back to where it once was and beyond - based on how the world changes. The ec

## Prep the "Stance" label

In [210]:
training_arguments['label'] = (training_arguments['Stance'] == 'against').astype(int)
validation_arguments['label'] = (validation_arguments['Stance'] == 'against').astype(int)
test_arguments['label'] = (test_arguments['Stance'] == 'against').astype(int)

In [211]:
training_arguments

,Argument ID,Conclusion,Stance,Premise,label
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,0
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,0
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,1
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,1
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...,1
...,...,...,...,...,...
5388,E08016,The EU should integrate the armed forces of it...,in favor of,"On the one hand, we have Russia killing countl...",0
5389,E08017,Food whose production has been subsidized with...,in favor of,The subsidies were originally intended to ensu...,0
5390,E08018,Food whose production has been subsidized with...,in favor of,These products come mainly from large enterpri...,0
5391,E08019,Food whose production has been subsidized with...,in favor of,Subsidies often make farmers in recipient coun...,0


In [212]:
for x in training_semlabels:
    print(training_semlabels[x].value_counts())
    print()

A01002    1
A27361    1
A27372    1
A27371    1
A27369    1
         ..
A21207    1
A21206    1
A21205    1
A21204    1
E08020    1
Name: Argument ID, Length: 5393, dtype: int64

0    4405
1     988
Name: Self-direction: thought, dtype: int64

0    3998
1    1395
Name: Self-direction: action, dtype: int64

0    5146
1     247
Name: Stimulation, dtype: int64

0    5221
1     172
Name: Hedonism, dtype: int64

0    3881
1    1512
Name: Achievement, dtype: int64

0    4783
1     610
Name: Power: dominance, dtype: int64

0    4768
1     625
Name: Power: resources, dtype: int64

0    5011
1     382
Name: Face, dtype: int64

0    3393
1    2000
Name: Security: personal, dtype: int64

0    3665
1    1728
Name: Security: societal, dtype: int64

0    4825
1     568
Name: Tradition, dtype: int64

0    4216
1    1177
Name: Conformity: rules, dtype: int64

0    5186
1     207
Name: Conformity: interpersonal, dtype: int64

0    4998
1     395
Name: Humility, dtype: int64

0    4061
1    1332
Name: B

In [213]:
def generate_pairwise_input(feature_dataset: pd.DataFrame):
    premises = feature_dataset['Premise'].to_list()
    conclusions = feature_dataset['Conclusion'].to_list()
    labels = feature_dataset['label'].to_list()
    
    return premises, conclusions, labels

## Prep premises, conslusions, and stance labels

In [214]:
train_premises, train_conclusions, train_labels = generate_pairwise_input(training_arguments)
val_premises, val_conclusions, val_labels = generate_pairwise_input(validation_arguments)
# test_premises, test_conclusions, test_labels = generate_pairwise_input(test_arguments)

In [215]:
test_premises = test_arguments["Premise"].to_list()
test_conclusions = test_arguments["Conclusion"].to_list()
test_labels = test_arguments["label"].to_list()

## One hot encode the stance label for Neural net architecture

In [216]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

train_labels_ohe = ohe.fit_transform(np.asarray(train_labels).reshape(-1,1)).astype(int)
val_labels_ohe = ohe.transform(np.asarray(val_labels).reshape(-1,1)).astype(int)
test_labels_ohe = ohe.transform(np.asarray(test_labels).reshape(-1,1)).astype(int)

In [217]:
len(train_labels_ohe)

5393

In [218]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def chunk_multi(lst1, lst2, n):
    for i in range(0, len(lst1), n):
        yield lst1[i: i + n], lst2[i: i + n]

## Batch the Train, Validation, and Test sets

In [219]:
batch_size = 200

tokenizer = BatchTokenizer()

train_input_batches = [b for b in chunk_multi(train_premises, train_conclusions, batch_size)]
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]

val_input_batches = [b for b in chunk_multi(val_premises, val_conclusions, batch_size)]
val_input_batches = [tokenizer(*batch) for batch in val_input_batches]

test_input_batches = [b for b in chunk_multi(test_premises, test_conclusions, batch_size)]
test_input_batches = [tokenizer(*batch) for batch in test_input_batches]

In [220]:
def encode_multi_labels(labels) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor

    Args:
        labels (List[int]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    return torch.LongTensor(labels)

In [222]:
train_label1_batches = [b for b in chunk(train_labels_ohe, 200)]
train_label1_batches = [encode_multi_labels(batch) for batch in train_label1_batches]

val_label1_batches = [b for b in chunk(val_labels_ohe, 200)]
val_label1_batches = [encode_multi_labels(batch) for batch in val_label1_batches]

test_label1_batches = [b for b in chunk(test_labels_ohe, 200)]
test_label1_batches = [encode_multi_labels(batch) for batch in test_label1_batches]

In [223]:
train_label1_batches[0].shape # check to ensure correct size

torch.Size([200, 2])

### Selecting which labels to predict

In the first cell, it is the labels with the more even representation, in the second cell it is all 20 labels

In [224]:
## Subset of labels
use_training_labels = training_semlabels.loc[:,["Self-direction: action", 
                                             "Security: personal", 
                                             "Universalism: concern"]].values.copy()
use_validation_labels = validation_semlabels.loc[:,["Self-direction: action", 
                                                 "Security: personal", 
                                                 "Universalism: concern"]].values.copy()

In [225]:
## All labels
# use_training_labels = training_semlabels.iloc[:,1:].values.copy()
# use_validation_labels = validation_semlabels.iloc[:,1:].values.copy()

In [226]:
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents)
    return list(torch.argmax(logits, axis=2).squeeze().numpy())

In [227]:
def predict_mult(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents)
    logits = logits.detach().numpy()
    mask_30  = (logits > 0.5)
    return mask_30.reshape(mask_30.shape[0], mask_30.shape[2]).astype(int)

# Naive Model 1

In this model, the data is first squeezed into two categories, then these predictions are added to the training sentences as extra features, which then goes through another sequence of layers to predict the labels from the 20 available.

In [228]:
import time

class NaiveClassifier(torch.nn.Module):
    def __init__(self, output_size1: int, output_size2: int, hidden_size: int):
        super().__init__()
        self.output_size1 = output_size1
        self.output_size2 = output_size2
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("prajjwal1/bert-small")
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        
        # create LSTM hidden layer
        self.hidden_layer = torch.nn.LSTM(self.bert_hidden_dimension, 
                                         self.hidden_size)
        # use ReLU for input to first classifier
        self.relu = torch.nn.ReLU()
        self.classifier1 = torch.nn.Linear(self.hidden_size, self.output_size1)
        # Log softmax for binary classification (are premise/conslusion in support)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
        # Sigmoid for multi-label prediction
        self.sigmoid = torch.nn.Sigmoid()
        
        self.hidden_layer2 = torch.nn.LSTM(self.bert_hidden_dimension + 2,
                                          self.hidden_size)
        self.classifier2 = torch.nn.Linear(self.hidden_size, self.output_size2)


    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)

        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        curr_batch = encoded_sequence['pooler_output'].shape[0]
        pooler_output = encoded_sequence['pooler_output'][:].reshape((curr_batch,
                                                                      1, 
                                                                      self.bert_hidden_dimension))
        
        return pooler_output

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        # print(encoded_sents.shape)
        output = self.hidden_layer(encoded_sents)
        # print(output[0].shape)
        output = self.relu(output[0])
        # print(output.shape)
        output = self.classifier1(output)
        # print(output.shape)
        # print(output[:5])
        first_res = self.log_softmax(output)
        # print(first_res.shape)
        new_data = torch.cat((encoded_sents, first_res), 2)
        output2 = self.hidden_layer2(new_data)
        output2 = self.relu(output2[0])
        output2 = self.classifier2(output2)
        second_res = self.log_softmax(output2)
        return second_res

In [229]:
import random
from tqdm import tqdm

def training_loop_single_model(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
):
    print("Training...")
    loss_func = torch.nn.BCEWithLogitsLoss()
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features).squeeze(1)
            loss = loss_func(preds, labels.float())
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.numpy()))
        
        all_labels = np.asarray(all_labels)
        preds = np.zeros((len(all_preds), all_labels.shape[1]))
        for i, x in enumerate(all_preds):
            preds[i,x] = 1
        dev_f1 = f1_score(preds, all_labels, average='macro')
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model, all_preds

## Using 3 labels

The labels included here are "Self-direction: action", "Security: personal", and "Universalism: concern". These labels had more in-class examples than others, which is why they were chosen. 

Considering we were using 3 labels, it is assumed that a random classifier would return a $33\%$ F1 score. 

In [230]:
partial_train_semlabel_batches = [b for b in chunk(use_training_labels, 200)]
partial_train_semlabel_batches = [encode_multi_labels(batch) for batch in partial_train_semlabel_batches]

partial_val_semlabel_batches = [b for b in chunk(use_validation_labels, 200)]
partial_val_semlabel_batches = [encode_multi_labels(batch) for batch in partial_val_semlabel_batches]

In [231]:
# You can increase epochs if need be
epochs = 15
# TODO: Find a good learning rate
LR = 0.00001 / 2

possible_labels = train_labels_ohe.shape[1]
naive_model = NaiveClassifier(output_size1=possible_labels,
                              output_size2=partial_train_semlabel_batches[0].shape[1], 
                              hidden_size=25)
optimizer = torch.optim.AdamW(naive_model.parameters(), LR)

naive_model, all_preds = training_loop_single_model(
    epochs,
    train_input_batches,
    partial_train_semlabel_batches,
    val_input_batches,
    partial_val_semlabel_batches,
    optimizer,
    naive_model,
)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:48<00:00,  1.81s/it]


epoch 0, loss: 0.6589264339870877
Evaluating dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.37it/s]


...

Dev F1 0.4676237683569373


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:49<00:00,  1.83s/it]


epoch 14, loss: 0.5978190656061526
Evaluating dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.36it/s]

Dev F1 0.4727143341487068


### Final F1 score

The highest F1 score achieved for the partial label naive model was $49\%$, and the lowest was $36\%$.

## Using full labels

Now we will train the same model, but this time using all the labels

In [232]:
full_training_labels = training_semlabels.iloc[:,1:].values.copy()
full_validation_labels = validation_semlabels.iloc[:,1:].values.copy()

train_semlabel_batches = [b for b in chunk(full_training_labels, 200)]
train_semlabel_batches = [encode_multi_labels(batch) for batch in train_semlabel_batches]

val_semlabel_batches = [b for b in chunk(full_validation_labels, 200)]
val_semlabel_batches = [encode_multi_labels(batch) for batch in val_semlabel_batches]

### Adjust the activation layer

New activation layer is sigmoid, which will return values $\in[0,1]$ to be used for multilabel (get all labels above a threshold). This is reflected in the `predict_mult` function.

In [233]:
import time

class NaiveMultiClass(torch.nn.Module):
    def __init__(self, output_size1: int, output_size2: int, hidden_size: int):
        super().__init__()
        self.output_size1 = output_size1
        self.output_size2 = output_size2
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("prajjwal1/bert-small")
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        
        # create LSTM hidden layer
        self.hidden_layer = torch.nn.LSTM(self.bert_hidden_dimension, 
                                         self.hidden_size)
        # use ReLU for input to first classifier
        self.relu = torch.nn.ReLU()
        self.classifier1 = torch.nn.Linear(self.hidden_size, self.output_size1)
        # Log softmax for binary classification (are premise/conslusion in support)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
        # Sigmoid for multi-label prediction
        self.sigmoid = torch.nn.Sigmoid()
        
        self.hidden_layer2 = torch.nn.LSTM(self.bert_hidden_dimension + 2,
                                          self.hidden_size)
        self.classifier2 = torch.nn.Linear(self.hidden_size, self.output_size2)


    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)

        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        curr_batch = encoded_sequence['pooler_output'].shape[0]
        pooler_output = encoded_sequence['pooler_output'][:].reshape((curr_batch,
                                                                      1, 
                                                                      self.bert_hidden_dimension))
        
        return pooler_output

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        # print(encoded_sents.shape)
        output = self.hidden_layer(encoded_sents)
        # print(output[0].shape)
        output = self.relu(output[0])
        # print(output.shape)
        output = self.classifier1(output)
        # print(output.shape)
        # print(output[:5])
        first_res = self.log_softmax(output)
        # print(first_res.shape)
        new_data = torch.cat((encoded_sents, first_res), 2)
        output2 = self.hidden_layer2(new_data)
        output2 = self.relu(output2[0])
        output2 = self.classifier2(output2)
        second_res = self.sigmoid(output2)
        return second_res

In [249]:
import random
from tqdm import tqdm

def training_loop_single_model(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
):
    print("Training...")
    loss_func = torch.nn.BCEWithLogitsLoss()
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features).squeeze(1)

            loss = loss_func(preds, labels.float())
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.numpy()))
        
        all_labels = np.asarray(all_labels)
        preds = np.zeros((len(all_preds), all_labels.shape[1]))
        for i, x in enumerate(all_preds):
            preds[i,x] = 1
        dev_f1 = f1_score(preds, all_labels, average='macro')
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model, all_preds

In [250]:
epochs = 15
LR = 5e-6

possible_labels = train_labels_ohe.shape[1]
naive_model2 = NaiveMultiClass(output_size1=possible_labels,
                              output_size2=train_semlabel_batches[0].shape[1], 
                              hidden_size=25)
optimizer = torch.optim.AdamW(naive_model2.parameters(), LR)

naive_model2, all_preds = training_loop_single_model(
    epochs,
    train_input_batches,
    train_semlabel_batches,
    val_input_batches,
    val_semlabel_batches,
    optimizer,
    naive_model2,
)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training...


  4%|█████▎                                                                                                                                         | 1/27 [00:04<01:51,  4.30s/it]

<class 'torch.Tensor'>
torch.Size([])


  7%|██████████▌                                                                                                                                    | 2/27 [00:05<01:05,  2.61s/it]

...

 26%|█████████████████████████████████████                                                                                                          | 7/27 [00:16<00:42,  2.14s/it]

<class 'torch.Tensor'>
torch.Size([])


 30%|██████████████████████████████████████████▎                                                                                                    | 8/27 [00:17<00:41,  2.19s/it]

<class 'torch.Tensor'>
torch.Size([])


KeyboardInterrupt: 

# Dual Model Approach

In this approach I will attempt to use two models, the first will be a model that predicts if a conclusion supports or is against its premise. From here, the second model will take as input the same premise/conclusions, but will use the labels from the first model to inform the second model.



## Rough sketch of how the ideal system would work
![model_sketch](https://www.imgur.com/iYkVRq6.jpg)

In [264]:
import time

class NLIClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("prajjwal1/bert-small")
        # self.bert = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity",)
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size.
        self.hidden_layer = torch.nn.LSTM(self.bert_hidden_dimension, 
                                         self.hidden_size)

        self.relu = torch.nn.ReLU()
        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)


    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)

        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        curr_batch = encoded_sequence['pooler_output'].shape[0]
        pooler_output = encoded_sequence['pooler_output'][:].reshape((curr_batch,
                                                                      1, 
                                                                      self.bert_hidden_dimension))
        
        return pooler_output

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output[0])
        output = self.classifier(output)
        output = self.log_softmax(output) #[200, 1, 2]
        return output #self.log_softmax(output)

In [289]:
import time

class semClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertForPreTraining.from_pretrained("textattack/bert-base-uncased-yelp-polarity",
                                                                  output_hidden_states=True)
        self.bert.config.num_labels = self.output_size
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size.
        self.hidden_layer = torch.nn.LSTM(self.bert_hidden_dimension, 
                                         self.hidden_size)

        self.relu = torch.nn.ReLU()
        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.sigmoid = torch.nn.Sigmoid()


    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # print(encoded_sequence['prediction_logits'].shape)
        # # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # for x in encoded_sequence['hidden_states']:
        #     print(x.shape)
        # curr_batch = encoded_sequence['hidden_states'][0].shape[0]
        # ret_val = encoded_sequence['hidden_states'][:,].reshape((curr_batch,
        #                                                               1, 
        #                                                               self.bert_hidden_dimension))
        
        # return the last hidden state's 
        # print(encoded_sequence['logits'].shape)
        return encoded_sequence['hidden_states'][-1][:,-1,:]

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        # print(len(encoded_sents))
        # print(encoded_sents.shape)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output[0])
        output = self.classifier(output) 
        return self.sigmoid(output)

In [290]:
import random
from tqdm import tqdm
from copy import deepcopy

def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents)
    return list(torch.argmax(logits, axis=1).squeeze().numpy())

def training_loop_dual_model(
    num_epochs,
    train_features,
    train_labels1,
    train_labels2,
    dev_sents,
    dev_labels1,
    dev_labels2,
    optimizer1,
    optimizer2,
    model1,
    model2,
):
    # print("Training...")
    loss_func = torch.nn.BCEWithLogitsLoss()
    nli_batches = list(zip(train_features, train_labels1))
    sem_batches = list(zip(train_features, train_labels2))
    random.shuffle(nli_batches)
    random.shuffle(sem_batches)
    for i in range(num_epochs):
        model_1_losses = []
        model_1_preds = []
        print("Training Model 1...")
        # first train the model to predict whether or not a 
        for features, labels in tqdm(nli_batches):
            # Empty the dynamic computation graph
            optimizer1.zero_grad()
            preds = model1(features).squeeze(1)
            # print(preds.shape)
            loss = loss_func(preds, labels.float())
            fw_preds = list(torch.argmax(preds, axis=1).numpy())
            model_1_preds.append(fw_preds)
            
            # Backpropogate the loss through our model
            loss.backward()
            optimizer1.step()
            model_1_losses.append(loss.item())
        
        print(f"epoch {i}, model 1 loss: {sum(model_1_losses)/len(model_1_losses)}")
        
        print("Training Model 2...")
        model_2_losses = []
        for (features, labels), m1_preds in tqdm(zip(sem_batches, model_1_preds), total=len(sem_batches)):
            # Empty the dynamic computation graph
            optimizer2.zero_grad()
            these_features = deepcopy(features)
            these_features['next_sentence_label'] = m1_preds
            preds = model2(these_features).squeeze(1)
            loss = loss_func(preds, labels.float())
            # Backpropogate the loss through our model
            loss.backward()
            optimizer2.step()
            model_2_losses.append(loss.item())
        
        print(f"epoch {i}, model 2 loss: {sum(model_2_losses)/len(model_2_losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating model 1 dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels1), total=len(dev_sents)):
            pred = predict(model1, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.numpy()))
        
        all_labels = np.asarray(all_labels)
        preds = np.zeros((len(all_preds), all_labels.shape[1]))
        for i, x in enumerate(all_preds):
            preds[i,x] = 1
        dev_f1 = f1_score(np.asarray(preds), np.asarray(all_labels), average='macro')
        print(f"Model 1 Dev F1 {dev_f1}")
        
        print("Evaluating model 2 dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels2), total=len(dev_sents)):
            pred = predict(model2, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.numpy()))
        
        all_labels = np.asarray(all_labels)
        preds = np.zeros((len(all_preds), all_labels.shape[1]))
        for i, x in enumerate(all_preds):
            preds[i,x] = 1
        dev_f1 = f1_score(np.asarray(preds), np.asarray(all_labels), average='macro')
        print(f"Model 2 Dev F1 {dev_f1}")
        
        
        
    # Return the trained model
    return model2, preds

In [291]:
# You can increase epochs if need be
epochs = 15
# TODO: Find a good learning rate
LR = 0.00001

possible_labels1 = train_labels_ohe.shape[1]
model1 = NLIClassifier(output_size=possible_labels1, 
                       hidden_size=25)
optimizer1 = torch.optim.AdamW(model1.parameters(), LR)

possible_labels2 = partial_train_semlabel_batches[0].shape[1]
model2 = semClassifier(output_size=possible_labels2,
                       hidden_size=40)
optimizer2 = torch.optim.AdamW(model2.parameters(), LR)

# validation_input_batches = [b for b in chunk_multi(val_premises, val_conclusions, 200)]
# # Tokenize + encode
# validation_input_batches = [tokenizer(*batch) for batch in validation_input_batches]
# validation_batch_labels = [b for b in chunk(val_labels, 200)]
# validation_batch_labels = [encode_labels(batch) for batch in validation_batch_labels]


model, all_preds = training_loop_dual_model(
    epochs,
    train_input_batches,
    train_label1_batches,
    partial_train_semlabel_batches,
    val_input_batches,
    val_label1_batches,
    partial_val_semlabel_batches,
    optimizer1,
    optimizer2,
    model1,
    model2,
)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at textattack/bert-base-uncased-yelp-polarity were not used when initializing BertF

Training Model 1...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:47<00:00,  1.76s/it]


epoch 0, model 1 loss: 0.7479404674635993
Training Model 2...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [05:24<00:00, 12.02s/it]


...

epoch 14, model 2 loss: 0.748737410262779
Evaluating model 1 dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.32it/s]


Model 1 Dev F1 0.35815842924847663
Evaluating model 2 dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:54<00:00,  5.43s/it]

Model 2 Dev F1 0.312348178533633


In [292]:
# You can increase epochs if need be
epochs = 15
# TODO: Find a good learning rate
LR = 5e-6

possible_labels1 = train_labels_ohe.shape[1]
model1 = NLIClassifier(output_size=possible_labels1, 
                       hidden_size=25)
optimizer1 = torch.optim.AdamW(model1.parameters(), LR)

possible_labels2 = train_semlabel_batches[0].shape[1]
model2 = semClassifier(output_size=possible_labels2,
                       hidden_size=40)
optimizer2 = torch.optim.AdamW(model2.parameters(), LR)

# validation_input_batches = [b for b in chunk_multi(val_premises, val_conclusions, 200)]
# # Tokenize + encode
# validation_input_batches = [tokenizer(*batch) for batch in validation_input_batches]
# validation_batch_labels = [b for b in chunk(val_labels, 200)]
# validation_batch_labels = [encode_labels(batch) for batch in validation_batch_labels]


model, all_preds = training_loop_dual_model(
    epochs,
    train_input_batches,
    train_label1_batches,
    train_semlabel_batches,
    val_input_batches,
    val_label1_batches,
    val_semlabel_batches,
    optimizer1,
    optimizer2,
    model1,
    model2,
)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at textattack/bert-base-uncased-yelp-polarity were not used when initializing BertF

Training Model 1...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:47<00:00,  1.75s/it]


epoch 0, model 1 loss: 0.750938406697026
Training Model 2...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [05:19<00:00, 11.84s/it]


...

epoch 14, model 2 loss: 0.8745663850395767
Evaluating model 1 dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.35it/s]


Model 1 Dev F1 0.35815842924847663
Evaluating model 2 dev...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:49<00:00,  4.99s/it]

Model 2 Dev F1 0.016379690949227373


## Test set on models

In [524]:
all_test_preds = []
for batch in test_input_batches:
    preds = predict(model, batch)
    all_test_preds.extend(preds)
    
test_preds = np.zeros((len(all_test_preds), 20))
for i, x in enumerate(all_test_preds):
    test_preds[i,x] = 1

In [534]:
res = pd.DataFrame(test_preds.astype(int), columns=training_labels.iloc[:,1:].columns,index=test_arguments['Argument ID'])
res.to_csv('./results/test_results.csv')

# Smart Model Approach

In this approach, instead of trying to predict if a conclusion is a continuation of the premise, I will just use a pretrained bert model and tell it myself, then use the model as previous

In [171]:
from transformers import BertForNextSentencePrediction

In [162]:
class NextSequenceBatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
    
    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token
    
    def __call__(self, prem_batch: List[str], hyp_batch: List[str], labels) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # The two sentences deimited by the [SEP] token.
        enc = self.hf_tokenizer(
            prem_batch,
            hyp_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc

In [173]:
tokenizer = BatchTokenizer()

train_input_batches = [b for b in chunk_multi(train_premises, train_conclusions, batch_size)]
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]

val_input_batches = [b for b in chunk_multi(val_premises, val_conclusions, batch_size)]
val_input_batches = [tokenizer(*batch) for batch in val_input_batches]

test_input_batches = [b for b in chunk_multi(test_premises, test_conclusions, batch_size)]
test_input_batches = [tokenizer(*batch) for batch in test_input_batches]

In [ ]:
training_arguments['label'] = (training_arguments['Stance'] == 'in favor of').astype(int)
validation_arguments['label'] = (validation_arguments['Stance'] == 'in favor of').astype(int)
test_arguments['label'] = (test_arguments['Stance'] == 'in favor of').astype(int)

In [433]:
import time

class SmartClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertForNextSentencePrediction.from_pretrained("prajjwal1/bert-small")
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size.
        self.hidden_layer = torch.nn.LSTM(self.bert_hidden_dimension, 
                                         self.hidden_size)

        self.relu = torch.nn.ReLU()
        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
        self.sigmoid = torch.nn.Sigmoid()


    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # print(encoded_sequence['prediction_logits'].shape)
        # # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # curr_batch = encoded_sequence['pooler_output'].shape[0]
        # ret_val = encoded_sequence['hidden_states'][:].reshape((curr_batch,
        #                                                               1, 
        #                                                               self.bert_hidden_dimension))
        
        return encoded_sequence['hidden_states']

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        print(len(encoded_sents))
        print(encoded_sents[0].shape)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output[0])
        output = self.classifier(output)
        return self.sigmoid(output)

2

In [514]:
# import numpy as np


# def precision(predicted_labels, true_labels, which_label=1):
#     """
#     Precision is True Positives / All Positives Predictions
#     """
#     pred_which = np.array(predicted_labels) == which_label
#     true_which = np.array(true_labels) == which_label
#     denominator = np.sum(pred_which)
#     if denominator:
#         return np.sum(np.logical_and(pred_which, true_which))/denominator
#     else:
#         return 0.


# def recall(predicted_labels, true_labels, which_label=1):
#     """
#     Recall is True Positives / All Positive Labels
#     """
#     pred_which = np.array(predicted_labels) == which_label
#     true_which = np.array(true_labels) == which_label
#     denominator = np.sum(true_which)
#     if denominator:
#         return np.sum(np.logical_and(pred_which, true_which))/denominator
#     else:
#         return 0.


# def f1_score(
#     predicted_labels: List[int],
#     true_labels: List[int],
#     which_label: int
# ):
#     """
#     F1 score is the harmonic mean of precision and recall
#     """
#     P = precision(predicted_labels, true_labels, which_label=which_label)
#     R = recall(predicted_labels, true_labels, which_label=which_label)
#     if P and R:
#         return 2*P*R/(P+R)
#     else:
#         return 0.


# def macro_f1_score(
#     predicted_labels: List[int],
#     true_labels: List[int],
#     possible_labels: List[int]
# ):
#     scores = [f1_score(predicted_labels, true_labels, l) for l in possible_labels]
#     # Macro, so we take the uniform avg.
#     return sum(scores) / len(scores)